#<a href='https://ahmad-almosallam.github.io/Machine-Learning-projects/' style='color:red;'>You Can test the model here</a>

# 1- importing the necessary libraries

In [61]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.base import clone

# 2- Gathering, cleaning, assessing data 

In [5]:
df = pd.read_csv("edu-data.csv")
df.sample(1)

,gender,NationalITy,PlaceofBirth,StageID,GradeID,SectionID,Topic,Semester,Relation,raisedhands,VisITedResources,AnnouncementsView,Discussion,ParentAnsweringSurvey,ParentschoolSatisfaction,StudentAbsenceDays,Class
462,M,Iraq,Iraq,MiddleSchool,G-08,A,History,F,Father,72,87,72,89,Yes,Good,Above-7,M


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   gender                    480 non-null    object
 1   NationalITy               480 non-null    object
 2   PlaceofBirth              480 non-null    object
 3   StageID                   480 non-null    object
 4   GradeID                   480 non-null    object
 5   SectionID                 480 non-null    object
 6   Topic                     480 non-null    object
 7   Semester                  480 non-null    object
 8   Relation                  480 non-null    object
 9   raisedhands               480 non-null    int64 
 10  VisITedResources          480 non-null    int64 
 11  AnnouncementsView         480 non-null    int64 
 12  Discussion                480 non-null    int64 
 13  ParentAnsweringSurvey     480 non-null    object
 14  ParentschoolSatisfaction  

#3- Preprocessing

In [7]:
numeric_attr = ["raisedhands", "VisITedResources", "AnnouncementsView", "Discussion"]
scaler = MinMaxScaler()
df[numeric_attr] = scaler.fit_transform(df[numeric_attr])

## Separating the features and the target

In [8]:
fetures = df.loc[:,:"StudentAbsenceDays"]
target = df.loc[:,"Class"]

## Replacing the categorical columns with numbers

In [12]:
replace_gender = {'gender': {'F': 1, 'M': 2}}
replace_nat = {'NationalITy': {'KW': 1, 'lebanon': 2, 'Egypt': 3, 'SaudiArabia': 4,
                               'USA':5,'Jordan': 6, 'venzuela': 7, 'Iran': 8 , 'Tunis': 9 ,
                               'Morocco': 10,'Syria': 11,'Palestine': 12,'Iraq': 13,'Lybia': 14}}

replace_place = {'PlaceofBirth': {'KuwaIT': 1, 'lebanon': 2, 'Egypt': 3, 'SaudiArabia': 4
                                  , 'USA':5,'Jordan': 6, 'venzuela': 7, 'Iran': 8 , 'Tunis': 9 ,
                                  'Morocco': 10,'Syria': 11,'Iraq': 12,'Palestine': 13,'Lybia': 14}}
replace_stage = {'StageID': {'lowerlevel': 1, 'MiddleSchool': 2, 'HighSchool': 3}}
replace_grade = {'GradeID': {'G-02':1,'G-04':2, 'G-05':3,'G-06':4,'G-07':5,'G-08':6,'G-09':7,'G-10':8,'G-11':9,'G-12':10}}
replace_sec = {'SectionID': {'A': 1, 'B': 2, 'C': 3}}
replace_topic = {'Topic': {'Arabic': 1, 'Biology': 2,'Chemistry': 3,
                           'English': 4, 'French':5,'Geology': 6,
                           'History': 7, 'IT': 8 , 'Math': 9 ,
                           'Quran': 10,'Science': 11,'Spanish': 12}}
replace_sems = {'Semester': {'F': 1, 'S': 2}}
replace_rel = {'Relation': {'Father': 1, 'Mum': 2}}
replace_pas = {'ParentAnsweringSurvey': {'Yes': 1, 'No': 2}}
replace_pss = {'ParentschoolSatisfaction': {'Bad': 1, 'Good': 2}}
replace_sad = {'StudentAbsenceDays': {'Above-7': 1, 'Under-7': 2}}

In [13]:
fetures.replace(replace_gender,inplace=True)
fetures.replace(replace_nat,inplace=True)
fetures.replace(replace_place,inplace=True)
fetures.replace(replace_stage,inplace=True)
fetures.replace(replace_grade,inplace=True)
fetures.replace(replace_sec,inplace=True)
fetures.replace(replace_topic,inplace=True)
fetures.replace(replace_sems,inplace=True)
fetures.replace(replace_rel,inplace=True)
fetures.replace(replace_pas,inplace=True)
fetures.replace(replace_pss,inplace=True)
fetures.replace(replace_sad,inplace=True)

In [ ]:
fetures.sample(1)

,gender,NationalITy,PlaceofBirth,StageID,GradeID,SectionID,Topic,Semester,Relation,raisedhands,VisITedResources,AnnouncementsView,Discussion,ParentAnsweringSurvey,ParentschoolSatisfaction,StudentAbsenceDays
174,1,6,6,1,1,2,5,2,1,0.5,0.626263,0.744898,0.428571,1,1,1


In [14]:
replace_class =  {'L': 1, 'M': 2 , 'H':3}
target.replace(replace_class,inplace=True)
target

0      2
1      2
2      1
3      1
4      2
      ..
475    1
476    2
477    2
478    1
479    1
Name: Class, Length: 480, dtype: int64

In [15]:
y = target
X = fetures

# 4- Training and Evaluation Result

In [62]:
models = {}
models['SVC'] = SVC()
models['DecisionTreeClassifier'] = DecisionTreeClassifier()
models['LogisticRegression'] = LogisticRegression(max_iter = 480)

In [63]:
conf_matrix_list_of_arrays = {}
conf_matrix_list_of_arrays["SVC"] = []
conf_matrix_list_of_arrays["DecisionTreeClassifier"] = []
conf_matrix_list_of_arrays["LogisticRegression"] = []
accuracy_results = {"SVC" : {"train":0, "test":0},"DecisionTreeClassifier" : {"train":0, "test":0},"LogisticRegression" : {"train":0, "test":0}}
f1score_results = {"SVC" : {"train":0, "test":0},"DecisionTreeClassifier" : {"train":0, "test":0},"LogisticRegression" : {"train":0, "test":0}}
kf = KFold(n_splits=10)
for model in models:
  for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    cloned_model = clone(models[model])
    cloned_model.fit(X_train, y_train)
    predictions_test = cloned_model.predict(X_test)
    predictions_train = cloned_model.predict(X_train)
    accuracy_results[model]["train"] += accuracy_score(y_train,predictions_train)
        
    
    accuracy_results[model]["test"] += accuracy_score(y_test,predictions_test)
    
    
    f1score_results[model]["train"] += f1_score(y_train,predictions_train,average='macro')
    
    
    f1score_results[model]["test"] += f1_score(y_test,predictions_test,average='macro')

    conf_matrix = confusion_matrix(y_test, predictions_test)
    conf_matrix_list_of_arrays[model].append(conf_matrix)
  accuracy_results[model]["train"] = accuracy_results[model]["train"] / 10
  accuracy_results[model]["test"] = accuracy_results[model]["test"] / 10
  f1score_results[model]["train"] = f1score_results[model]["train"] / 10
  f1score_results[model]["test"] = f1score_results[model]["test"] / 10
  conf_matrix_list_of_arrays[model] = np.mean(conf_matrix_list_of_arrays[model], axis=0)
  print(model + ":")
  print("Accuracy:" , 'train: ', accuracy_results[model]["train"], '| test: ', accuracy_results[model]["test"])
  print("F1-score:" , 'train: ', f1score_results[model]["train"], '| test: ', f1score_results[model]["test"])
  print("Confusion Matrix:\n" , conf_matrix_list_of_arrays[model])
  print("---------------------------------------------------------")

SVC:
Accuracy: train:  0.6534722222222222 | test:  0.5541666666666668
F1-score: train:  0.6001804491782009 | test:  0.4512688717225039
Confusion Matrix:
 [[ 6.5  5.7  0.5]
 [ 1.9 17.7  1.5]
 [ 0.  11.8  2.4]]
---------------------------------------------------------
DecisionTreeClassifier:
Accuracy: train:  1.0 | test:  0.6354166666666667
F1-score: train:  1.0 | test:  0.6357921475026476
Confusion Matrix:
 [[ 9.5  3.   0.2]
 [ 3.1 12.4  5.6]
 [ 0.1  5.5  8.6]]
---------------------------------------------------------
LogisticRegression:
Accuracy: train:  0.788888888888889 | test:  0.7437499999999999
F1-score: train:  0.7956598735226803 | test:  0.7439810991429413
Confusion Matrix:
 [[10.7  2.   0. ]
 [ 2.5 14.6  4. ]
 [ 0.   3.8 10.4]]
---------------------------------------------------------


# 5- SVM Hyperparameters Tuning 

In [25]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, 
                                                    random_state = 0)
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 384 samples.
Testing set has 96 samples.


In [49]:
param_svc = {'C':[0.1, 1, 10, 100, 1000],'gamma':[1,0.1,0.01,0.001,0.0001], 'kernel':['linear','rbf']}
grid_svc = GridSearchCV(SVC(),param_svc)
grid_svc.fit(X_train,y_train)
grid_svc_pred_test = grid_svc.predict(X_test)
grid_svc_pred_train = grid_svc.predict(X_train)

In [55]:
accuracy_results_train = accuracy_score(y_train,grid_svc_pred_train)
accuracy_results_test = accuracy_score(y_test,grid_svc_pred_test)


f1score_results_train = f1_score(y_train,grid_svc_pred_train,average='macro')
f1score_results_test= f1_score(y_test,grid_svc_pred_test,average='macro')


conf_matrix = confusion_matrix(y_test,grid_svc_pred_test)

In [60]:
print("SVC accuracy after grid search ", "train ->", accuracy_results_train)
print("SVC accuracy after grid search ", "test ->", accuracy_results_test)
print("SVC F1-score after grid search ", "train ->", f1score_results_train)
print("SVC F1-score after grid search ", "test ->", f1score_results_test)
print("SVC F1-score after grid search ", "\nconfusion matrix ->\n", conf_matrix)

SVC accuracy after grid search  train -> 0.7864583333333334
SVC accuracy after grid search  test -> 0.71875
SVC F1-score after grid search  train -> 0.7922821371319366
SVC F1-score after grid search  test -> 0.7290603299164027
SVC F1-score after grid search  
confusion matrix ->
 [[22  1  0]
 [ 4 31 10]
 [ 0 12 16]]
